# Grouping data in pandas

You can group and aggregate data in pandas in ways that will be familiar if you've ever done a pivot table in Excel or a GROUP BY statement in SQL. In this notebook we'll use the eel import data that lives at `../data/eels.csv`.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/eels.csv')

In [3]:
df.head()

,year,month,country,product,kilos,dollars
0,2010,1,CHINA,EELS FROZEN,49087,393583
1,2010,1,JAPAN,EELS FRESH,263,7651
2,2010,1,TAIWAN,EELS FROZEN,9979,116359
3,2010,1,VIETNAM,EELS FRESH,1938,10851
4,2010,1,VIETNAM,EELS FROZEN,21851,69955


### `groupby()`

Let's group the data by country and sum the kilos for each country.

If this were a pivot table, we'd drag the `country` column into Rows and the `kilos` column into Values, then summarize by Sum.

If this were SQL, we might write something like:

```sql
SELECT country, sum(kilos)
FROM table
GROUP BY country
ORDER BY 2 desc
```

Let's do the same thing in pandas using [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html):

- Select our two columns of interest (`country` and `kilos`)
- Call the `groupby()` method on the grouping column (`country`)
- Call the `sum()` method
- Sort by kilos descending

In [4]:
df[['country', 'kilos']].groupby('country').sum().sort_values('kilos', ascending=False)

,kilos
country,
CHINA,15965996
VIETNAM,637737
TAIWAN,442740
JAPAN,361364
CANADA,346075
SOUTH KOREA,243540
THAILAND,137556
PORTUGAL,41453
PAKISTAN,22453


### Value counts

If all you need to do is count the occurrences of a value in a column, you can use the `value_counts()` method for a Series.

In our eel data, every row is one month's of shipments of a particular eel product from one country. In how many months is mainland China represented, period? Of those, how many times did its monthly exports to the U.S. exceed 25,000 kilos?

Our steps:
- Get the value_counts of the country data and peep China's total
- Filter the data to get just shipments over 25,000 kilos, then get the value_counts on country again, peep China's total

In [ ]:
df.country.value_counts().sort_values(ascending=False).head()

In [ ]:
df[df['kilos'] > 25000].country.value_counts().sort_values(ascending=False)

### Pivot tables

Now we want to get the total kilos by country by year. We could use `groupby()` again, but pass it multiple columns. We'd get something like this:

In [ ]:
df[['country', 'year', 'kilos']].groupby(['country', 'year']).sum()

... which is fine, but (I think) there's a more intuitive way to look at this data: using the [`pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) method.

If we were making this pivot table in Excel, we would drag `country` to Rows, `kilos` to Values and `year` to Columns. But we're gonna do it in pandas. We're gonna hand the `pivot_table()` method four things:
- A reference to the data frame you're pivoting (`df`)
- The `index` column -- what to group your data by (`index='country'`)
- The `columns` column -- the second grouping factor (`columns='year'`)
- The `aggfunc` -- what function to use to aggregate the data; the default is to use an average, but we'll use Python's built-in `sum` function

Then we'll sort the results by the latest year of data -- 2017 -- and fill null values with zeroes.

In [7]:
pd.pivot_table(df,
               index='country',
               columns='year',
               values='kilos',
               aggfunc=sum).sort_values(2017, ascending=False) \
                           .fillna(0)

year,2010,2011,2012,2013,2014,2015,2016,2017
country,,,,,,,,
CHINA,372397.0,249232.0,1437392.0,1090135.0,1753140.0,4713882.0,4578546.0,1771272.0
TAIWAN,73842.0,0.0,53774.0,39752.0,83478.0,48272.0,99535.0,44087.0
SOUTH KOREA,42929.0,41385.0,28146.0,27353.0,37708.0,8386.0,14729.0,42904.0
JAPAN,1326.0,2509.0,32255.0,105758.0,40177.0,69699.0,71748.0,37892.0
THAILAND,2866.0,5018.0,9488.0,4488.0,15110.0,41771.0,26931.0,31884.0
VIETNAM,63718.0,155488.0,118063.0,100828.0,38112.0,36859.0,96179.0,28490.0
CANADA,13552.0,24968.0,110796.0,44455.0,31546.0,28619.0,68568.0,23571.0
PORTUGAL,2081.0,3672.0,2579.0,2041.0,7215.0,8013.0,9105.0,6747.0
PANAMA,0.0,0.0,0.0,11849.0,0.0,0.0,0.0,974.0
